# SDG 6.6.1: Analysis and Reporting

## Extracting GSWE data

In [1]:
import ee
from GSWE_reporting import extract_gswe
ee.Authenticate()

Enter verification code: 4/1AfDhmriiFFT4jr2l2TNj5BtJZO8WT_Kv5VbGQFHHKELQVaxLe_aMJ8FKTZ0

Successfully saved authorization token.


In [2]:
ee.Initialize()  

In [3]:
geometry = ee.Geometry.Polygon(
        [[[-12.598544729822379, 61.78863494939058],
          [-12.598544729822379, 49.00174346741333],
          [3.749111520177621, 49.00174346741333],
          [3.749111520177621, 61.78863494939058]]])

In [4]:
extract_gswe('JRC/GSW1_2/YearlyHistory', geometry, 1984, 2020, 'GSWE_exports')

GSWE files are being extracted to GSWE_exports in your Google Drive


---
## Preparing HydroBASIN boundary

In [1]:
import geopandas as gpd
from shapely.ops import unary_union
from GSWE_reporting import clip_basin_to_boundary

In [2]:
eu_hydrobasin = gpd.read_file('./boundaries/hybas_eu_lev06_v1c.shp')
uk_boundary = gpd.read_file('./boundaries/Countries_December_2018_Boundaries_UK_BFC.shp') 

In [3]:
target_crs = ('+proj=utm +zone=30 +ellps=WGS84 +datum=WGS84 +units=m +no_defs ')
osgb_proj4 = ('+proj=tmerc +lat_0=49 +lon_0=-2 +k=0.9996012717 +x_0=400000 +y_0=-100000 +ellps=airy +datum=OSGB36 +units=m +no_defs')
uk_boundary.crs = osgb_proj4  

In [4]:
hydro_clipped = clip_basin_to_boundary(eu_hydrobasin, uk_boundary, target_crs)

union national boundary


In [5]:
hydro_clipped = hydro_clipped.drop(hydro_clipped.columns[1:-1], axis=1) 
hydro_clipped.to_file('./boundaries/UK_hydrobasin_UTM.shp')

---
## Preparing GSWE outputs

In [1]:
import geopandas as gpd
from GSWE_reporting import get_gswe_paths, reproject_GSWE

In [2]:
gsw_files = get_gswe_paths('./GSW_output/', '.tif')

In [3]:
reproject_GSWE(gsw_files, '+proj=utm +zone=30 +ellps=WGS84 +datum=WGS84 +units=m +no_defs ')

---
## Calculating Water Extent

In [1]:
import pandas as pd
import altair as alt
import geopandas as gpd
from GSWE_reporting import get_gswe_paths, surface_water_extent

In [2]:
gsw_file_path_list = get_gswe_paths('./Reprojected/', 'UTM.tif')

In [3]:
hydro_basin = gpd.read_file('./boundaries/UK_hydrobasin_UTM.shp')

In [4]:
water_extent = surface_water_extent(gsw_file_path_list, hydro_basin)

Calculating water extent in 1984
Calculating water extent in 1985
Calculating water extent in 1986
Calculating water extent in 1987
Calculating water extent in 1988
Calculating water extent in 1989
Calculating water extent in 1990
Calculating water extent in 1991
Calculating water extent in 1992
Calculating water extent in 1993
Calculating water extent in 1994
Calculating water extent in 1995
Calculating water extent in 1996
Calculating water extent in 1997
Calculating water extent in 1998
Calculating water extent in 1999
Calculating water extent in 2000
Calculating water extent in 2001
Calculating water extent in 2002
Calculating water extent in 2003
Calculating water extent in 2004
Calculating water extent in 2005
Calculating water extent in 2006
Calculating water extent in 2007
Calculating water extent in 2008
Calculating water extent in 2009
Calculating water extent in 2010
Calculating water extent in 2011
Calculating water extent in 2012
Calculating water extent in 2013
Calculatin

In [5]:
hydro_basin['area'] = hydro_basin.area / 1000000
water_extent = hydro_basin[['HYBAS_ID', 'area']].merge(water_extent, on='HYBAS_ID', how = 'left')
water_extent[['% Ephemeral', '% Seasonal', '% Permanent']] = water_extent[['Ephemeral', 'Seasonal', 'Permanent']].div(water_extent['area'], axis=0)*100

### National Water Extent

In [6]:
water_extent['HYBAS_ID'] = water_extent["HYBAS_ID"].apply(str)
water_type_by_year = water_extent.groupby(['Year'], as_index=False).sum() 

In [7]:
water_type_melt = pd.melt(water_type_by_year[water_type_by_year['Year']!='baseline'], id_vars=['Year'],
                          value_vars=['Ephemeral', 'Seasonal', 'Permanent'],
                          var_name='Type', value_name = 'Extent')

alt.Chart(water_type_melt).mark_bar(opacity=0.7).encode(
    x=alt.X('Year:N', title = 'Year'),
    y=alt.Y('sum(Extent)', title = 'Water Extent (km\N{SUPERSCRIPT TWO})'),
    color=alt.Color('Type',
        scale = alt.Scale(domain=['Permanent', 'Ephemeral', 'Seasonal'],
                          range=['#1f78b4', '#b2df8a', '#a6cee3']),
        title = 'Water Type'),
    order=alt.Order('sum(Type)',sort='ascending'),
    tooltip = ['Year:N', 
               alt.Tooltip('Extent:Q', title = 'Extent')]
).interactive()

alt.Chart(...)

In [8]:
water_type_by_year['HYBAS_ID'] = 'Total'
water_type_by_year[['% Ephemeral', '% Seasonal', '% Permanent']] = water_type_by_year[['Ephemeral', 'Seasonal', 'Permanent']].div(water_type_by_year['area'], axis=0)*100
water_type_by_basin = water_extent.append(water_type_by_year, sort=False, ignore_index = True)
water_type_by_basin.to_csv('./water_type_w_total.csv', header = True, index = False) 

---

## Percentage Extent Change

In [9]:
def filter_years(df, from_year, to_year, basin_id, water_type):
    filtered = df[(df['HYBAS_ID'] == str(basin_id)) & \
                  (df['Year'] >= from_year) & \
                  (df['Year'] <= to_year)][water_type]
    return filtered

In [10]:
water_extent_df = pd.read_csv('./water_type_w_total.csv')
water_extent_df = water_extent_df[['HYBAS_ID', 'Year', 'Ephemeral', 'Seasonal', 'Permanent']]
hydrobasin_extent_df = water_extent_df[water_extent_df['Year']!='baseline']
hydrobasin_extent_df[['Year']] = hydrobasin_extent_df['Year'].astype('int')
baseline = water_extent_df[water_extent_df['Year']=='baseline']

<ipython-input-10-df9f2165ccad>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hydrobasin_extent_df[['Year']] = hydrobasin_extent_df['Year'].astype('int')
/Users/guyhannay/Library/Caches/pypoetry/virtualenvs/sdg-661-reporting-5LNc9BCe-py3.8/lib/python3.8/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


In [11]:
cols = ['HYBAS_ID', 'Water Type','2010-2014 av. water extent', '2010-2014 % change from baseline',
        '2015-2019 av. water extent', '2015-2019 % change from baseline']
water_extent_change = pd.DataFrame(columns = cols)

for basin_id in baseline['HYBAS_ID']:
    for water_type in baseline.columns[2:]:
        baseline_extent = baseline[(baseline['HYBAS_ID'] == basin_id)][water_type].values[0]
        
        filtered_2010_2014 = filter_years(hydrobasin_extent_df, 2010, 2014, basin_id, water_type)
        filtered_2015_2019 = filter_years(hydrobasin_extent_df, 2015, 2019, basin_id, water_type)
        
        filtered_2010_2014 = filtered_2010_2014.mean()
        filtered_2015_2019 = filtered_2015_2019.mean()

        if baseline_extent == 0:
            perc_change_2010_2014 = 'NA'
            perc_change_2015_2019 = 'NA'

        else:
            perc_change_2010_2014 = (baseline_extent-filtered_2010_2014)/baseline_extent*100
            perc_change_2015_2019 = (baseline_extent-filtered_2015_2019)/baseline_extent*100
            
        values = [basin_id, water_type, filtered_2010_2014, perc_change_2010_2014, 
                  filtered_2015_2019, perc_change_2015_2019]
            
        hydrobasin_extent_change = pd.DataFrame([values], columns = cols)        
        water_extent_change = pd.concat([water_extent_change, hydrobasin_extent_change])

In [12]:
baseline_melt = pd.melt(baseline, id_vars=['HYBAS_ID'], value_vars=['Ephemeral', 'Seasonal', 'Permanent']
                        , value_name = 'Baseline', var_name = 'Water Type')
water_extent_w_baseline = baseline_melt.merge(water_extent_change, how = 'left',
                                   on = ['HYBAS_ID', 'Water Type'])

In [13]:
water_extent_w_baseline.to_csv('water_extent_change.csv', index =False)